# Guider mode image reduction
Craig Lage - 07-Oct-24

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from astropy.table import Table

In [ ]:
filename = "/home/c/cslage/u/ComCam/guider_mode/MC_C_20241005_000148_R04_SG1_guider.fits"
hdu_list = fits.open(filename, do_not_scale_image_data=True)

In [ ]:
hdu_list[0].header

In [ ]:
hdu_list[3].header

In [ ]:
expId = 2024100500148
dirName = f"/home/c/cslage/u/ComCam/guider_mode/movie_{expId}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}.mp4"
print(movieName)

# Build the individual frames
fig = plt.figure(figsize=(8, 8))
nStamps = int(hdu_list[0].header['N_STAMPS'])
for n in range(nStamps):
    data = hdu_list[2 * n + 2].data
    plt.imshow(data, interpolation='nearest')
    plt.title(f"ComCam guider mode {expId}, Frame {n+1}", fontsize=12)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()


In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
data = hdu_list[5].data[0][0]
data.shape

In [ ]:
23850/9/50

In [ ]:
data = (hdu_list[5].data['rawStamp'][0]).astype('>u4')
data.byteswap(inplace=True)
xor = 0x20000
size = 50 * 53
out = np.zeros([16, size], dtype=int)
image_out = np.zeros([16, 50,50], dtype=int)
for n in range(size):
    res = ''
    for i in range(9):
        d = data[(size-n)*9-i-1]
        d = format(d, '#034b')
        d = d.split('b')[1]
        res += d
    for i in range(16):
        bin_value = res[i*18:(i+1)*18]
        int_value = int(bin_value, 2)
        final_value = int_value ^ xor
        out[i,n] = final_value  
for i in range(16):
    reshaped = out[i,:].reshape(50,53)
    image_out[i,:,:] = np.fliplr(reshaped[:,0:50])

In [ ]:
plt.imshow?

In [ ]:
plt.imshow(image_out[13], interpolation='nearest', origin='lower')
plt.colorbar()

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(10,10))
plt.subplots_adjust(wspace=.7, hspace=-0.2)
for i in range(4):
    for j in range(4):
        n = i + 4 * j
        axs[i][j].set_title(f"segment {n}")
        im = axs[i][j].imshow(image_out[n], interpolation='nearest', origin='lower')
        divider = make_axes_locatable(axs[i][j])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)


In [ ]:
data = hdu_list[6].data
plt.imshow(data, interpolation='nearest')
plt.colorbar()
data.shape

In [ ]:
for i in range(16):
    diff = image_out[i] - data
    print(i, np.max(diff), np.min(diff))


In [ ]:
a = image_out[13]
i,j = np.unravel_index(a.argmax(), a.shape)
print(i,j)
print(a[i,j])
b = data
i,j = np.unravel_index(b.argmax(), b.shape)
print(i,j)
print(b[i,j])
a = image_out[13]
i,j = np.unravel_index(a.argmin(), a.shape)
print(i,j)
print(a[i,j])
b = data
i,j = np.unravel_index(b.argmin(), b.shape)
print(i,j)
print(b[i,j])

In [ ]:
data = (hdu_list[5].data['rawStamp'][0]).astype('>u4')
d = data[0]
print(d)
print(bin(d))
d = format(d, '#034b')
d = d.split('b')[1]
print(d)
last_18 = d[14::]
print(last_18)
print(int(last_18, 2))

In [ ]:
format(0x4, '#06b')

In [ ]:
filename = "/home/c/cslage/u/ComCam/guider_mode/MC_C_20241005_000148_R04_SG1_guider.fits"
hdu_list = fits.open(filename)
data = (hdu_list[5].data['rawStamp'][0]).astype('>u4')
data.byteswap(inplace=True)
mask = 0x3ffff
xor = 0x20000
print(data[0])
print(hex(data[0]))
print(bin(data[0]))
print(bin(mask))
first_18 = data[0] & mask
print(bin(first_18))
print(first_18)
xored = first_18 ^ xor
print(bin(xored))
print(xored)


In [ ]:
from sys import byteorder
print(byteorder)

In [ ]:
mask = 0x3ffff
print(mask)
print(bin(mask))

In [ ]:
size = 2
for n in range(size):
    for i in range(9):
        print(n, i, (size-n)*9-i-1)